# 🐳 Diagnostic et Résolution des Erreurs de Build Docker

Ce notebook diagnostique l'erreur de construction d'image Docker rencontrée sur GitHub Actions lors de l'installation des packages Python optimisés.

## 🚨 Erreur rencontrée
```
ERROR: failed to build: failed to solve: process "/bin/bash -o pipefail -c pip3 install --upgrade pip setuptools wheel &&     pip3 install         einops safetensors jupyterlab ipywidgets         flash-attn --no-build-isolation         torch-audio-addons         triton         xformers &&     rm -rf /root/.cache/pip" did not complete successfully: exit code: 1
```

## 1. 📊 Analyser les logs d'erreur Docker

Les packages problématiques typiques dans ce type d'erreur :

In [ ]:
# Packages susceptibles de causer des problèmes de compilation
problematic_packages = {
    'flash-attn': {
        'compilation_required': True,
        'cuda_dependency': True,
        'memory_intensive': True,
        'common_issues': ['CUDA toolkit manquant', 'Mémoire insuffisante', 'GCC version incompatible']
    },
    'xformers': {
        'compilation_required': True,
        'cuda_dependency': True,
        'memory_intensive': True,
        'common_issues': ['PyTorch version incompatible', 'CUDA version mismatch']
    },
    'triton': {
        'compilation_required': True,
        'cuda_dependency': True,
        'memory_intensive': False,
        'common_issues': ['LLVM dependency', 'Architecture non supportée']
    },
    'torch-audio-addons': {
        'compilation_required': False,
        'cuda_dependency': False,
        'memory_intensive': False,
        'common_issues': ['Package inexistant ou obsolète']
    }
}

print("🔍 Analyse des packages problématiques:")
for package, info in problematic_packages.items():
    print(f"\n📦 {package}:")
    print(f"  • Compilation requise: {info['compilation_required']}")
    print(f"  • Dépendance CUDA: {info['cuda_dependency']}")
    print(f"  • Intensif en mémoire: {info['memory_intensive']}")
    print(f"  • Problèmes courants: {', '.join(info['common_issues'])}")

## 2. 🔍 Vérifier la compatibilité des dépendances Python

Analysons les versions et compatibilités requises :

In [ ]:
# Matrice de compatibilité PyTorch/CUDA pour les packages
compatibility_matrix = {
    'pytorch_2.8.0_cuda_12.9': {
        'flash-attn': {
            'supported': True,
            'version_range': '>=2.5.0',
            'notes': 'Nécessite CUDA 11.6+, compatible avec PyTorch 2.8'
        },
        'xformers': {
            'supported': True,
            'version_range': '>=0.0.23',
            'notes': 'Version compatible avec PyTorch 2.8 pas encore stable'
        },
        'triton': {
            'supported': True,
            'version_range': '>=2.1.0',
            'notes': 'Inclus avec PyTorch 2.8'
        },
        'torch-audio-addons': {
            'supported': False,
            'version_range': 'N/A',
            'notes': 'Package non trouvé - probablement obsolète'
        }
    }
}

print("📋 Matrice de compatibilité PyTorch 2.8.0 + CUDA 12.9:")
for package, info in compatibility_matrix['pytorch_2.8.0_cuda_12.9'].items():
    status = "✅" if info['supported'] else "❌"
    print(f"\n{status} {package}:")
    print(f"  • Version: {info['version_range']}")
    print(f"  • Notes: {info['notes']}")

## 3. 🧪 Tester l'installation des packages individuellement

Script pour identifier le package problématique :

In [ ]:
# Script de test d'installation séquentielle
test_script = """
#!/bin/bash
set -e

echo "🔧 Test d'installation séquentielle des packages..."

# Packages de base (peu de risques)
BASIC_PACKAGES="einops safetensors jupyterlab ipywidgets"

# Packages à risque (compilation CUDA)
RISKY_PACKAGES="flash-attn triton xformers"

# Package suspect
SUSPECT_PACKAGE="torch-audio-addons"

echo "📦 Installation des packages de base..."
pip install $BASIC_PACKAGES
echo "✅ Packages de base installés avec succès"

echo "🔥 Test du package suspect..."
if pip install $SUSPECT_PACKAGE; then
    echo "✅ $SUSPECT_PACKAGE installé avec succès"
else
    echo "❌ $SUSPECT_PACKAGE a échoué - probable cause de l'erreur"
fi

echo "⚡ Installation des packages à risque un par un..."
for package in $RISKY_PACKAGES; do
    echo "Installation de $package..."
    if pip install $package --no-build-isolation; then
        echo "✅ $package installé avec succès"
    else
        echo "❌ $package a échoué"
        exit 1
    fi
done
"""

print("📝 Script de test généré:")
print(test_script)

# Sauvegarder le script
with open('/Users/abscons/Documents/Docker/Runpod/Runpod_Github/test-packages.sh', 'w') as f:
    f.write(test_script)

print("\n💾 Script sauvegardé dans: test-packages.sh")

## 4. 🔧 Optimiser le Dockerfile pour la construction

Stratégies d'optimisation :

In [ ]:
# Générer un Dockerfile optimisé
optimized_dockerfile_section = """
# Installation des packages Python optimisés pour Blackwell - Version Robuste
RUN pip3 install --upgrade pip setuptools wheel && \\
    # Installation des packages de base (sans risque)
    pip3 install einops safetensors jupyterlab ipywidgets && \\
    # Installation de flash-attn avec gestion d'erreur
    (pip3 install flash-attn --no-build-isolation || \\
     echo "⚠️ flash-attn installation failed, continuing without it") && \\
    # Installation de xformers avec version spécifique
    (pip3 install xformers || \\
     pip3 install xformers==0.0.23.post1 || \\
     echo "⚠️ xformers installation failed, continuing without it") && \\
    # Triton est inclus avec PyTorch 2.8, installation optionnelle
    (pip3 install triton || echo "⚠️ triton installation failed, using PyTorch built-in version") && \\
    # Suppression du package problématique torch-audio-addons
    # Ce package semble obsolète ou inexistant
    rm -rf /root/.cache/pip
"""

print("🐳 Section Dockerfile optimisée:")
print(optimized_dockerfile_section)

# Alternative avec installation en deux étapes
two_stage_installation = """
# Alternative: Installation en deux étapes pour réduire les risques

# Étape 1: Packages de base
RUN pip3 install --upgrade pip setuptools wheel && \\
    pip3 install einops safetensors jupyterlab ipywidgets && \\
    rm -rf /root/.cache/pip

# Étape 2: Packages d'optimisation (avec gestion d'erreur)
RUN set +e && \\
    echo "Installation des optimisations GPU..." && \\
    pip3 install flash-attn --no-build-isolation && echo "✅ flash-attn installé" || echo "❌ flash-attn échoué" && \\
    pip3 install xformers && echo "✅ xformers installé" || echo "❌ xformers échoué" && \\
    pip3 install triton && echo "✅ triton installé" || echo "❌ triton échoué" && \\
    rm -rf /root/.cache/pip && \\
    set -e
"""

print("\n🔄 Alternative en deux étapes:")
print(two_stage_installation)

## 5. 🛡️ Implémenter des stratégies de fallback

Création de stratégies alternatives pour assurer le succès du build :

In [ ]:
# Stratégies de fallback pour chaque package
fallback_strategies = {
    'flash-attn': [
        'pip install flash-attn --no-build-isolation',
        'pip install flash-attn==2.5.8 --no-build-isolation',
        'pip install flash-attn --no-deps --no-build-isolation',
        'echo "Continuing without flash-attn"'
    ],
    'xformers': [
        'pip install xformers',
        'pip install xformers==0.0.23.post1',
        'pip install xformers --index-url https://download.pytorch.org/whl/cu121',
        'echo "Continuing without xformers"'
    ],
    'triton': [
        'pip install triton',
        'pip install triton==2.1.0',
        'echo "Using PyTorch built-in triton"'
    ],
    'torch-audio-addons': [
        'echo "Package torch-audio-addons obsolète, ignoré"'
    ]
}

# Générer un script de fallback
fallback_script = """
#!/bin/bash
# Script d'installation avec stratégies de fallback

install_with_fallback() {
    local package_name=$1
    shift
    local strategies=("$@")
    
    echo "📦 Installation de $package_name..."
    
    for strategy in "${strategies[@]}"; do
        echo "🔄 Tentative: $strategy"
        if eval $strategy; then
            echo "✅ $package_name installé avec succès"
            return 0
        else
            echo "❌ Stratégie échouée, tentative suivante..."
        fi
    done
    
    echo "⚠️ Toutes les stratégies ont échoué pour $package_name"
    return 1
}

# Installation des packages de base
pip install --upgrade pip setuptools wheel
pip install einops safetensors jupyterlab ipywidgets

# Installation avec fallback pour chaque package problématique
"""

for package, strategies in fallback_strategies.items():
    if package != 'torch-audio-addons':  # On ignore ce package obsolète
        strategy_array = ' '.join([f'"{s}"' for s in strategies])
        fallback_script += f"""
install_with_fallback "{package}" {strategy_array}
"""

fallback_script += """
# Nettoyage final
rm -rf /root/.cache/pip
echo "✅ Installation terminée"
"""

print("🛡️ Script de fallback généré:")
print(fallback_script)

# Sauvegarder le script
with open('/Users/abscons/Documents/Docker/Runpod/Runpod_Github/install-with-fallback.sh', 'w') as f:
    f.write(fallback_script)

print("\n💾 Script sauvegardé dans: install-with-fallback.sh")

## 🎯 Recommandations immédiates

### 1. **Problème identifié probable**: `torch-audio-addons`
Ce package semble **obsolète ou inexistant**. Il faut le supprimer.

### 2. **Solution immédiate**: Modifier le Dockerfile
Remplacer la section d'installation par une version robuste avec gestion d'erreurs.

### 3. **Test recommandé**: 
Utiliser le script `test-packages.sh` généré pour identifier précisément le package défaillant.

### 4. **Stratégie long terme**:
Implémenter le système de fallback pour assurer la robustesse du build.

In [ ]:
# Génération du correctif immédiat pour le Dockerfile
immediate_fix = """
# CORRECTIF IMMÉDIAT - Remplacer dans le Dockerfile principal:

# AVANT (version actuelle avec erreur):
RUN pip3 install --upgrade pip setuptools wheel && \\
    pip3 install \\
        einops safetensors jupyterlab ipywidgets \\
        flash-attn --no-build-isolation \\
        torch-audio-addons \\
        triton \\
        xformers && \\
    rm -rf /root/.cache/pip

# APRÈS (version corrigée):
RUN pip3 install --upgrade pip setuptools wheel && \\
    pip3 install \\
        einops safetensors jupyterlab ipywidgets && \\
    pip3 install flash-attn --no-build-isolation || echo "flash-attn installation failed" && \\
    pip3 install triton || echo "triton installation failed" && \\
    pip3 install xformers || echo "xformers installation failed" && \\
    rm -rf /root/.cache/pip

# CHANGEMENTS:
# ❌ Supprimé: torch-audio-addons (package obsolète/inexistant)
# ✅ Ajouté: Gestion d'erreur pour les packages d'optimisation
# ✅ Séparé: Installation en deux étapes pour isolation
"""

print("🔧 CORRECTIF IMMÉDIAT:")
print(immediate_fix)

# Sauvegarder le correctif
with open('/Users/abscons/Documents/Docker/Runpod/Runpod_Github/dockerfile-fix.md', 'w') as f:
    f.write(immediate_fix)

print("\n💾 Correctif sauvegardé dans: dockerfile-fix.md")
print("\n🚀 Prochaines étapes:")
print("1. Appliquer le correctif au Dockerfile")
print("2. Tester le build localement")
print("3. Commiter et pousser les changements")
print("4. Surveiller le build GitHub Actions")

# 🚨 Diagnostic des services RunPod non accessibles

## Problème rencontré
Tous les ports HTTP configurés sur RunPod ne sont pas accessibles :
- Port 3000: HTTP Service **Not Ready**
- Port 8188: HTTP Service **Not Ready** 
- Port 8888: HTTP Service **Not Ready**

## Configuration RunPod actuelle
```
HTTP Ports:
- HTTP Service: 8188
- HTTP Service: 8888  
- HTTP Service: 3000

TCP Ports:
- TCP Port: 22
- TCP Port: 443

Environment Variables:
- HF_TOKEN: {{ RUNPOD_SECRET_HF_TOKEN }}
- JUPYTER_PASSWORD: {{ RUNPOD_SECRET_JUPYTER_PASSWORD }}
```

In [ ]:
# Analyse des causes possibles des services non accessibles
import subprocess
import os

potential_issues = {
    "script_demarrage": {
        "description": "Le script /start.sh ne démarre pas correctement",
        "symptoms": ["Tous les services down", "Container se lance mais services inactifs"],
        "solutions": ["Vérifier logs du container", "Vérifier permissions du script", "Vérifier syntaxe bash"]
    },
    "nginx_config": {
        "description": "Configuration nginx incorrecte ou service nginx non démarré",
        "symptoms": ["Port 3000 non accessible", "Proxy vers 8188 non fonctionnel"],
        "solutions": ["Vérifier nginx.conf", "Démarrer nginx manuellement", "Vérifier logs nginx"]
    },
    "comfyui_startup": {
        "description": "ComfyUI ne démarre pas sur le port 8188",
        "symptoms": ["Port 8188 non accessible", "Service ComfyUI en erreur"],
        "solutions": ["Vérifier logs ComfyUI", "Vérifier dépendances Python", "Vérifier workspace"]
    },
    "jupyter_config": {
        "description": "JupyterLab non configuré ou démarré",
        "symptoms": ["Port 8888 non accessible"],
        "solutions": ["Vérifier variable JUPYTER_PASSWORD", "Démarrer jupyter manuellement"]
    },
    "permissions": {
        "description": "Problèmes de permissions sur les fichiers/répertoires",
        "symptoms": ["Scripts non exécutables", "Erreurs d'accès fichiers"],
        "solutions": ["Vérifier chmod +x", "Vérifier ownership des fichiers"]
    },
    "workspace_volume": {
        "description": "Volume /workspace non monté ou inaccessible",
        "symptoms": ["Erreurs lors de l'initialisation", "ComfyUI ne trouve pas les fichiers"],
        "solutions": ["Vérifier montage volume", "Vérifier permissions volume"]
    }
}

print("🔍 ANALYSE DES CAUSES POSSIBLES")
print("=" * 50)

for issue_id, issue_info in potential_issues.items():
    print(f"\n❌ {issue_info['description']}")
    print(f"   Symptômes: {', '.join(issue_info['symptoms'])}")
    print(f"   Solutions: {', '.join(issue_info['solutions'])}")

print("\n" + "=" * 50)
print("🎯 PRIORITÉ DE VÉRIFICATION:")
print("1. Script de démarrage (/start.sh)")
print("2. Configuration nginx")
print("3. Démarrage ComfyUI")
print("4. Permissions et volumes")

In [ ]:
# Analyse des scripts de démarrage
print("🔍 ANALYSE DES SCRIPTS DE DÉMARRAGE")
print("=" * 50)

# Vérification de la logique de démarrage
startup_flow = {
    "1. pre_start.sh": {
        "taches": [
            "Copie ComfyUI vers /workspace/ComfyUI",
            "Installation des extensions",
            "Configuration nginx.conf",
            "DÉMARRAGE ComfyUI sur port 8188"
        ],
        "problemes_potentiels": [
            "Volume /workspace non monté",
            "Permissions insuffisantes",
            "ComfyUI ne démarre pas",
            "Dépendances Python manquantes"
        ]
    },
    "2. start.sh": {
        "taches": [
            "Appel de pre_start.sh",
            "Démarrage nginx (port 3000 -> proxy vers 8188)",
            "Configuration SSH",
            "Démarrage Jupyter (port 8888)",
            "Sleep infinity"
        ],
        "problemes_potentiels": [
            "pre_start.sh échoue",
            "nginx.conf invalide",
            "Proxy nginx non fonctionnel",
            "JUPYTER_PASSWORD non défini"
        ]
    }
}

for step, info in startup_flow.items():
    print(f"\n📋 {step}")
    print("   Tâches:")
    for task in info["taches"]:
        print(f"     • {task}")
    print("   Problèmes potentiels:")
    for problem in info["problemes_potentiels"]:
        print(f"     ❌ {problem}")

print("\n🎯 DIAGNOSTICS RECOMMANDÉS:")
print("1. Vérifier les logs du container RunPod")
print("2. Se connecter en SSH et vérifier les processus")
print("3. Tester manuellement chaque service")
print("4. Vérifier les fichiers de logs générés")

In [ ]:
# Génération d'un script de diagnostic pour RunPod
diagnostic_script = """#!/bin/bash
echo "🔍 DIAGNOSTIC RUNPOD - Services non accessibles"
echo "========================================================"

echo ""
echo "📋 1. VÉRIFICATION DES PROCESSUS"
echo "----------------------------------------"
echo "ComfyUI (port 8188):"
if pgrep -f "python.*main.py.*8188" > /dev/null; then
    echo "✅ ComfyUI est en cours d'exécution"
    ps aux | grep "python.*main.py" | grep -v grep
else
    echo "❌ ComfyUI n'est PAS en cours d'exécution"
fi

echo ""
echo "Nginx:"
if pgrep nginx > /dev/null; then
    echo "✅ Nginx est en cours d'exécution"
    ps aux | grep nginx | grep -v grep
else
    echo "❌ Nginx n'est PAS en cours d'exécution"
fi

echo ""
echo "Jupyter:"
if pgrep -f "jupyter" > /dev/null; then
    echo "✅ Jupyter est en cours d'exécution"
    ps aux | grep jupyter | grep -v grep
else
    echo "❌ Jupyter n'est PAS en cours d'exécution"
fi

echo ""
echo "📋 2. VÉRIFICATION DES PORTS"
echo "----------------------------------------"
netstat -tlnp | grep -E ":(3000|8188|8888|22)" || echo "Aucun port actif trouvé"

echo ""
echo "📋 3. VÉRIFICATION DES FICHIERS DE LOG"
echo "----------------------------------------"
echo "Log ComfyUI:"
if [ -f "/var/log/comfyui.log" ]; then
    echo "✅ Log ComfyUI trouvé"
    echo "Dernières lignes:"
    tail -n 10 /var/log/comfyui.log
else
    echo "❌ Log ComfyUI non trouvé"
fi

echo ""
echo "Log Jupyter:"
if [ -f "/jupyter.log" ]; then
    echo "✅ Log Jupyter trouvé"
    echo "Dernières lignes:"
    tail -n 5 /jupyter.log
else
    echo "❌ Log Jupyter non trouvé"
fi

echo ""
echo "📋 4. VÉRIFICATION DE LA CONFIGURATION"
echo "----------------------------------------"
echo "Répertoire ComfyUI:"
ls -la /workspace/ComfyUI/ | head -n 10

echo ""
echo "Configuration nginx:"
if [ -f "/workspace/ComfyUI/nginx.conf" ]; then
    echo "✅ nginx.conf trouvé"
else
    echo "❌ nginx.conf manquant"
fi

echo ""
echo "Variables d'environnement importantes:"
echo "JUPYTER_PASSWORD: ${JUPYTER_PASSWORD:-'Non défini'}"
echo "HF_TOKEN: ${HF_TOKEN:-'Non défini'}"

echo ""
echo "📋 5. TEST DE CONNECTIVITÉ LOCALE"
echo "----------------------------------------"
echo "Test port 8188 (ComfyUI):"
curl -s -o /dev/null -w "%{http_code}" http://localhost:8188 || echo "Échec connexion"

echo ""
echo "Test port 3000 (nginx):"
curl -s -o /dev/null -w "%{http_code}" http://localhost:3000 || echo "Échec connexion"

echo ""
echo "========================================================"
echo "✅ Diagnostic terminé"
"""

# Sauvegarder le script
with open('/Users/abscons/Documents/Docker/Runpod/Runpod_Github/runpod-diagnostic.sh', 'w') as f:
    f.write(diagnostic_script)

print("📝 Script de diagnostic généré: runpod-diagnostic.sh")
print("🚀 Pour l'utiliser sur RunPod:")
print("1. Se connecter en SSH au pod")
print("2. Copier le script et le rendre exécutable:")
print("   chmod +x runpod-diagnostic.sh")
print("3. Exécuter: ./runpod-diagnostic.sh")
print("\nLe script va analyser:")
print("• Processus en cours")
print("• Ports ouverts") 
print("• Fichiers de logs")
print("• Configuration")
print("• Connectivité locale")

# 🎯 Solutions potentielles identifiées

## Problème probable #1: Volume /workspace non monté
**Symptôme**: Tous les services échouent car ComfyUI ne peut pas se copier vers /workspace/ComfyUI

**Solution RunPod**:
```bash
# Dans la configuration du template RunPod, ajouter:
Volume Mounts:
- Container Path: /workspace
- Volume Size: 50GB (ou plus selon besoins)
```

## Problème probable #2: Variables d'environnement manquantes
**Symptôme**: Jupyter ne démarre pas, services incomplets

**Solution RunPod**:
Vérifier que les variables d'environnement sont bien définies:
```
JUPYTER_PASSWORD = {{ RUNPOD_SECRET_JUPYTER_PASSWORD }}
HF_TOKEN = {{ RUNPOD_SECRET_HF_TOKEN }}
```

## Problème probable #3: Erreur dans pre_start.sh
**Symptôme**: pre_start.sh échoue, services ne démarrent pas

**Solution**: Voir les logs du container pour identifier l'erreur spécifique

In [ ]:
# Script de réparation automatique pour RunPod
repair_script = """#!/bin/bash
echo "🔧 RÉPARATION AUTOMATIQUE DES SERVICES RUNPOD"
echo "=============================================="

# Fonction pour démarrer ComfyUI manuellement
start_comfyui() {
    echo "🚀 Démarrage manuel de ComfyUI..."
    cd /workspace/ComfyUI
    
    # Arrêter toute instance précédente
    pkill -f "python.*main.py" 2>/dev/null || true
    
    # Créer le répertoire de logs
    mkdir -p /var/log
    
    # Démarrer ComfyUI
    nohup python main.py \\
        --listen \\
        --port 8188 \\
        --extra-model-paths-config /workspace/ComfyUI/extra_model_paths.yml \\
        --force-fp16 \\
        --use-split-cross-attention \\
        --enable-cors-header \\
        > /var/log/comfyui.log 2>&1 &
    
    sleep 3
    if pgrep -f "python.*main.py" > /dev/null; then
        echo "✅ ComfyUI démarré avec succès"
    else
        echo "❌ Échec du démarrage de ComfyUI"
        return 1
    fi
}

# Fonction pour démarrer nginx
start_nginx() {
    echo "🌐 Démarrage de nginx..."
    
    # Copier la configuration
    if [ -f "/workspace/ComfyUI/nginx.conf" ]; then
        cp /workspace/ComfyUI/nginx.conf /etc/nginx/nginx.conf
    else
        echo "❌ Configuration nginx manquante"
        return 1
    fi
    
    # Démarrer nginx
    service nginx start
    
    if pgrep nginx > /dev/null; then
        echo "✅ Nginx démarré avec succès"
    else
        echo "❌ Échec du démarrage de nginx"
        return 1
    fi
}

# Fonction pour démarrer Jupyter
start_jupyter() {
    if [ -n "$JUPYTER_PASSWORD" ]; then
        echo "📓 Démarrage de Jupyter..."
        
        pkill -f "jupyter" 2>/dev/null || true
        
        cd /workspace
        nohup jupyter lab --allow-root --no-browser --port=8888 --ip=* \\
            --ServerApp.token=$JUPYTER_PASSWORD --ServerApp.allow_origin=* \\
            --ServerApp.preferred_dir=/workspace &> /jupyter.log &
        
        sleep 2
        if pgrep -f "jupyter" > /dev/null; then
            echo "✅ Jupyter démarré avec succès"
        else
            echo "❌ Échec du démarrage de Jupyter"
        fi
    else
        echo "⚠️ JUPYTER_PASSWORD non défini, Jupyter non démarré"
    fi
}

# Vérifications préliminaires
echo "📋 Vérifications préliminaires..."

if [ ! -d "/workspace" ]; then
    echo "❌ Volume /workspace non monté!"
    echo "   Vérifiez la configuration du template RunPod"
    exit 1
fi

if [ ! -f "/workspace/ComfyUI/main.py" ]; then
    echo "⚠️ ComfyUI non trouvé, exécution du script de pré-démarrage..."
    bash /pre_start.sh
fi

# Démarrage des services
start_comfyui
start_nginx  
start_jupyter

echo ""
echo "✅ Réparation terminée!"
echo "📋 Vérifiez les services:"
echo "   - ComfyUI: http://localhost:8188"
echo "   - Interface (nginx): http://localhost:3000" 
echo "   - Jupyter: http://localhost:8888"
"""

# Sauvegarder le script de réparation
with open('/Users/abscons/Documents/Docker/Runpod/Runpod_Github/runpod-repair.sh', 'w') as f:
    f.write(repair_script)

print("🔧 Script de réparation généré: runpod-repair.sh")
print("📋 ACTIONS RECOMMANDÉES:")
print("1. Exécuter le diagnostic: ./runpod-diagnostic.sh")
print("2. Si problème détecté, exécuter: ./runpod-repair.sh")
print("3. Vérifier la configuration du template RunPod")
print("4. S'assurer que le volume /workspace est bien monté")

# 🚀 Plan d'action recommandé

## 🔥 Actions immédiates sur RunPod

### 1. **Vérification du volume /workspace**
Dans votre template RunPod, assurez-vous d'avoir :
```
Volume Mounts:
- Container Path: /workspace  
- Volume Size: 50GB minimum
```

### 2. **Connexion SSH et diagnostic**
```bash
# Se connecter au pod RunPod via SSH
# Puis exécuter les commandes suivantes:

# Copier et exécuter le script de diagnostic
wget https://raw.githubusercontent.com/Seven-94/Runpod_ComfyUI/main/runpod-diagnostic.sh
chmod +x runpod-diagnostic.sh
./runpod-diagnostic.sh

# Si problème détecté, exécuter la réparation
wget https://raw.githubusercontent.com/Seven-94/Runpod_ComfyUI/main/runpod-repair.sh  
chmod +x runpod-repair.sh
./runpod-repair.sh
```

### 3. **Vérification manuelle des services**
```bash
# Vérifier les processus
ps aux | grep -E "(python|nginx|jupyter)"

# Vérifier les ports  
netstat -tlnp | grep -E ":(3000|8188|8888)"

# Tester la connectivité locale
curl http://localhost:8188  # ComfyUI direct
curl http://localhost:3000  # Interface nginx
curl http://localhost:8888  # Jupyter
```

## 📋 Causes les plus probables

### **#1 Volume /workspace non monté** (90% probable)
- **Symptôme**: Tous les services échouent
- **Solution**: Configurer le volume dans le template RunPod

### **#2 Script pre_start.sh échoue** (80% probable) 
- **Symptôme**: ComfyUI ne démarre pas
- **Solution**: Vérifier les logs et exécuter manuellement

### **#3 Configuration nginx défaillante** (60% probable)
- **Symptôme**: Port 3000 non accessible
- **Solution**: Vérifier nginx.conf et redémarrer nginx

## ✅ Validation du succès

Une fois les corrections appliquées, vous devriez voir :
- **Port 8188**: ComfyUI interface direct ✅
- **Port 3000**: Interface ComfyUI via nginx ✅  
- **Port 8888**: JupyterLab (si JUPYTER_PASSWORD défini) ✅
- **Port 22**: SSH access ✅